# Курсовой проект для курса "Python для Data Science"

### Постановка задачи
**Материалы к проекту (файлы):**
train.csv
test.csv

**Задание:**
Используя данные из train.csv, построить
модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели предсказать
цены для квартир из файла test.csv.

**Целевая переменная:**
Price

**Метрика:**
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

### Реализация проекта
**Подключение библиотек**

In [2227]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

**Загрузка данных**

In [2228]:
DATASET_PATH = 'train.csv'
train = pd.read_csv(DATASET_PATH, sep = ',')
train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [2229]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

In [2230]:
train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [2231]:
DATASET_PATH = 'test.csv'
X_test = pd.read_csv(DATASET_PATH, sep = ',')
X_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


In [2232]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             5000 non-null   int64  
 1   DistrictId     5000 non-null   int64  
 2   Rooms          5000 non-null   float64
 3   Square         5000 non-null   float64
 4   LifeSquare     3959 non-null   float64
 5   KitchenSquare  5000 non-null   float64
 6   Floor          5000 non-null   int64  
 7   HouseFloor     5000 non-null   float64
 8   HouseYear      5000 non-null   int64  
 9   Ecology_1      5000 non-null   float64
 10  Ecology_2      5000 non-null   object 
 11  Ecology_3      5000 non-null   object 
 12  Social_1       5000 non-null   int64  
 13  Social_2       5000 non-null   int64  
 14  Social_3       5000 non-null   int64  
 15  Healthcare_1   2623 non-null   float64
 16  Helthcare_2    5000 non-null   int64  
 17  Shops_1        5000 non-null   int64  
 18  Shops_2 

**Уменьшение объема памяти, который занимает датасет**

In [2233]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [2234]:
train = reduce_mem_usage(train)

Memory usage of dataframe is 1.53 MB
Memory usage after optimization is: 0.49 MB
Decreased by 68.1%


In [2235]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             10000 non-null  int16   
 1   DistrictId     10000 non-null  int16   
 2   Rooms          10000 non-null  float32 
 3   Square         10000 non-null  float32 
 4   LifeSquare     7887 non-null   float32 
 5   KitchenSquare  10000 non-null  float32 
 6   Floor          10000 non-null  int8    
 7   HouseFloor     10000 non-null  float32 
 8   HouseYear      10000 non-null  int32   
 9   Ecology_1      10000 non-null  float32 
 10  Ecology_2      10000 non-null  category
 11  Ecology_3      10000 non-null  category
 12  Social_1       10000 non-null  int8    
 13  Social_2       10000 non-null  int16   
 14  Social_3       10000 non-null  int16   
 15  Healthcare_1   5202 non-null   float32 
 16  Helthcare_2    10000 non-null  int8    
 17  Shops_1        10000 non-null  i

In [2236]:
X_test = reduce_mem_usage(X_test)

Memory usage of dataframe is 0.72 MB
Memory usage after optimization is: 0.21 MB
Decreased by 70.3%


In [2237]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             5000 non-null   int16   
 1   DistrictId     5000 non-null   int16   
 2   Rooms          5000 non-null   float32 
 3   Square         5000 non-null   float32 
 4   LifeSquare     3959 non-null   float32 
 5   KitchenSquare  5000 non-null   float32 
 6   Floor          5000 non-null   int8    
 7   HouseFloor     5000 non-null   float32 
 8   HouseYear      5000 non-null   int16   
 9   Ecology_1      5000 non-null   float32 
 10  Ecology_2      5000 non-null   category
 11  Ecology_3      5000 non-null   category
 12  Social_1       5000 non-null   int8    
 13  Social_2       5000 non-null   int16   
 14  Social_3       5000 non-null   int16   
 15  Healthcare_1   2623 non-null   float32 
 16  Helthcare_2    5000 non-null   int8    
 17  Shops_1        5000 non-null   in

**Обзор количественных и номинативных переменных**

In [2238]:
train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315712,37.199596,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904419,1.319500,4.231300,214139.218750
std,4859.01902,43.587592,0.839523,21.058718,86.241112,28.561113,5.241148,6.776192,2.005003e+05,0.119026,17.532614,4006.799803,23.831875,1021.518982,1.493601,4.806341,92872.304688
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.777344
25%,4169.50000,20.000000,1.000000,41.774879,22.769833,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.628906
50%,8394.50000,36.000000,2.000000,52.513309,32.781261,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.648438
75%,12592.50000,75.000000,2.000000,65.900627,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.460938
max,16798.00000,209.000000,19.000000,641.065186,7480.592285,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.437500


In [2239]:
train.nunique()

Id               10000
DistrictId         205
Rooms                9
Square            9995
LifeSquare        7886
KitchenSquare       58
Floor               33
HouseFloor          44
HouseYear           97
Ecology_1          129
Ecology_2            2
Ecology_3            2
Social_1            51
Social_2           142
Social_3            30
Healthcare_1        79
Helthcare_2          7
Shops_1             16
Shops_2              2
Price             9996
dtype: int64

**Обработка пропусков**

Пропуски в Healthcare_1 заменим на среднее арифметическое

In [2240]:
Healthcare_1_mean = train["Healthcare_1"].mean()
print(Healthcare_1_mean)
train["Healthcare_1"].fillna(Healthcare_1_mean, inplace = True)

1142.9044189453125


In [2241]:
Healthcare_1_mean = X_test["Healthcare_1"].mean()
print(Healthcare_1_mean)
X_test["Healthcare_1"].fillna(Healthcare_1_mean, inplace = True)

1146.6572265625


Пропуски в признаке **LifeSquare** заменим на произведение **Square** и отношения среднего **LifeSquare** к среднему **Square**:

In [2242]:
#train
LifeSquare_mean = train["LifeSquare"].mean()
Square_mean = train["Square"].mean()
ratio_LifeSquare_mean_Square_mean = LifeSquare_mean/Square_mean
train["LifeSquare"].fillna((train['Square']*ratio_LifeSquare_mean_Square_mean), inplace = True)

In [2243]:
#test
LifeSquare_mean = X_test["LifeSquare"].mean()
Square_mean = X_test["Square"].mean()
ratio_LifeSquare_mean_Square_mean = LifeSquare_mean/Square_mean
X_test["LifeSquare"].fillna((X_test['Square']*ratio_LifeSquare_mean_Square_mean), inplace = True)

**Обработка выбросов**

In [2244]:
train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315712,37.763058,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.939575,1.319500,4.231300,214139.218750
std,4859.01902,43.587592,0.839523,21.058718,76.909637,28.561113,5.241148,6.776192,2.005003e+05,0.119026,17.532614,4006.799803,23.831875,736.732727,1.493601,4.806341,92872.304688
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.777344
25%,4169.50000,20.000000,1.000000,41.774879,24.763958,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,830.000000,0.000000,1.000000,153872.628906
50%,8394.50000,36.000000,2.000000,52.513309,33.583181,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1142.904419,1.000000,3.000000,192269.648438
75%,12592.50000,75.000000,2.000000,65.900627,45.178256,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1142.904419,2.000000,6.000000,249135.460938
max,16798.00000,209.000000,19.000000,641.065186,7480.592285,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.437500


Найдем выбросы с помощью квантилей

Обработка выбросов в признаке **Rooms**

In [2245]:
train['Rooms'].describe()

count    10000.000000
mean         1.890500
std          0.839523
min          0.000000
25%          1.000000
50%          2.000000
75%          2.000000
max         19.000000
Name: Rooms, dtype: float64

In [2246]:
Rooms_max_value = np.quantile(train['Rooms'],q=0.999)
Rooms_max_value

5.0

In [2247]:
Rooms_min_value = np.quantile(train['Rooms'],q=0.001)
Rooms_min_value

1.0

In [2248]:
condition = (train['Rooms'] > Rooms_max_value) | (train['Rooms'] < Rooms_min_value)
train[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
377,5927,57,10.0,59.056976,36.223072,10.0,22,22.0,2002,0.090799,B,B,74,19083,2,1142.904419,5,15,B,317265.312500
1397,12638,27,0.0,138.427689,136.215500,0.0,4,3.0,2016,0.075424,B,B,11,3097,0,1142.904419,0,0,B,268394.750000
1454,8491,1,19.0,42.006046,21.779287,7.0,17,17.0,2014,0.007122,B,B,1,264,0,1142.904419,0,1,B,78364.617188
1981,7917,27,0.0,212.932358,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,1142.904419,0,1,B,302211.250000
2170,14003,99,6.0,59.414333,38.702244,6.0,7,9.0,1969,0.033494,B,B,66,10573,1,1322.000000,3,8,B,229661.968750
2269,7317,27,0.0,41.790882,27.605154,0.0,13,0.0,1977,0.211401,B,B,9,1892,0,1142.904419,0,1,B,98129.976562
3911,770,28,0.0,49.483501,32.686546,0.0,16,0.0,2015,0.118537,B,B,30,6207,1,1183.000000,1,0,B,217009.343750
4366,456,6,0.0,81.491447,53.829540,0.0,4,0.0,1977,0.243205,B,B,5,1564,0,540.000000,0,0,B,212864.796875
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,1142.904419,0,0,B,126596.945312
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,B,B,43,8429,3,1142.904419,3,9,B,158998.109375


In [2249]:
train.loc[condition, 'Rooms'] = train['Rooms'].median()
train['Rooms'].describe()

count    10000.000000
mean         1.888400
std          0.811071
min          1.000000
25%          1.000000
50%          2.000000
75%          2.000000
max          5.000000
Name: Rooms, dtype: float64

Обработка выбросов в признаке **Square**

In [2250]:
train['Square'].describe()

count    10000.000000
mean        56.315712
std         21.058718
min          1.136859
25%         41.774879
50%         52.513309
75%         65.900627
max        641.065186
Name: Square, dtype: float64

In [2251]:
Square_max_value = np.quantile(train['Square'],q=0.999)
Square_max_value

183.38480256652883

In [2252]:
Square_min_value = np.quantile(train['Square'],q=0.001)
Square_min_value

5.128915897369385

In [2253]:
condition = (train['Square'] > Square_max_value) | (train['Square'] < Square_min_value)
train[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
652,3718,56,3.0,190.857681,107.376167,19.0,7,3.0,2007,0.000000,B,B,23,5067,4,1142.904419,2,6,B,338644.062500
1316,11526,27,1.0,4.633498,1.969969,1.0,18,1.0,1977,0.011654,B,B,4,915,0,1142.904419,0,0,B,107604.265625
1608,10202,6,1.0,2.596351,4.604943,1.0,3,25.0,2014,0.243205,B,B,5,1564,0,540.000000,0,0,B,137597.593750
1981,7917,27,2.0,212.932358,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,1142.904419,0,1,B,302211.250000
1982,5548,86,5.0,275.645294,233.949310,26.0,12,37.0,2011,0.161976,B,A,31,7010,5,4508.000000,3,7,B,455264.875000
3280,10527,27,1.0,4.380726,40.805836,1.0,10,17.0,2013,0.211401,B,B,9,1892,0,1142.904419,0,1,B,97560.718750
4262,28,9,2.0,604.705994,399.441254,1.0,17,18.0,1977,0.161532,B,B,25,5648,1,30.000000,2,4,B,187717.250000
4690,2307,102,1.0,409.425171,410.639740,10.0,4,4.0,2016,0.238617,B,B,26,3889,6,705.000000,3,6,B,90470.429688
4853,3224,27,2.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,B,B,2,469,0,1142.904419,0,0,B,126596.945312
4900,4504,27,3.0,4.390331,5.610772,1.0,8,19.0,2016,0.211401,B,B,9,1892,0,1142.904419,0,1,B,161379.062500


In [2254]:
train.loc[condition, 'Square'] = train['Square'].median()
train.Square.describe()

count    10000.000000
mean        56.106720
std         18.637201
min          5.129221
25%         41.789598
50%         52.513309
75%         65.870504
max        183.382278
Name: Square, dtype: float64

Обработка выбросов в признаке **LifeSquare**

In [2255]:
train['LifeSquare'].describe()

count    10000.000000
mean        37.763058
std         76.909637
min          0.370619
25%         24.763958
50%         33.583181
75%         45.178256
max       7480.592285
Name: LifeSquare, dtype: float64

In [2256]:
LifeSquare_max_value = np.quantile(train['LifeSquare'],q=0.999)
LifeSquare_max_value

170.7177769927987

In [2257]:
LifeSquare_min_value = np.quantile(train['LifeSquare'],q=0.001)
LifeSquare_min_value

1.168772310972214

In [2258]:
condition = (train['LifeSquare'] > LifeSquare_max_value) | (train['LifeSquare'] < LifeSquare_min_value)
train[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
26,4378,27,3.0,106.958870,0.641822,0.0,17,0.0,2018,0.072158,B,B,2,629,1,1142.904419,0,0,A,337299.875000
590,14990,23,2.0,48.449875,263.542023,5.0,6,5.0,1972,0.075779,B,B,6,1437,3,1142.904419,0,2,B,141780.234375
1641,11055,151,1.0,174.649521,174.837082,15.0,10,12.0,1958,0.236108,B,B,16,3893,27,1142.904419,3,10,B,232766.187500
1981,7917,27,2.0,52.513309,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,1142.904419,0,1,B,302211.250000
1982,5548,86,5.0,52.513309,233.949310,26.0,12,37.0,2011,0.161976,B,A,31,7010,5,4508.000000,3,7,B,455264.875000
2887,15402,27,2.0,58.521099,1.049867,1.0,4,17.0,2014,0.072158,B,B,2,629,1,1142.904419,0,0,A,142393.078125
3935,12198,6,1.0,41.939732,1.071193,1.0,12,17.0,2019,0.243205,B,B,5,1564,0,540.000000,0,0,B,127233.289062
4262,28,9,2.0,52.513309,399.441254,1.0,17,18.0,1977,0.161532,B,B,25,5648,1,30.000000,2,4,B,187717.250000
4268,6631,85,1.0,48.990532,1.104689,1.0,5,17.0,1977,0.072158,B,B,2,629,1,1142.904419,0,0,A,124761.835938
4328,16550,27,3.0,81.694420,7480.592285,1.0,9,17.0,2016,0.017647,B,B,2,469,0,1142.904419,0,0,B,217357.500000


In [2259]:
train.loc[condition, 'LifeSquare'] = train['LifeSquare'].median()
train.LifeSquare.describe()

count    10000.000000
mean        36.781746
std         16.589211
min          1.168791
25%         24.832663
50%         33.583183
75%         45.141671
max        170.713654
Name: LifeSquare, dtype: float64

In [2260]:
condition = (train.LifeSquare > train.Square)
train.loc[condition, 'LifeSquare'], train.loc[condition, 'Square'] = train['Square'], train['LifeSquare']

In [2261]:
train.LifeSquare.describe()

count    10000.000000
mean        36.664158
std         16.378716
min          1.168791
25%         24.827632
50%         33.583183
75%         45.054755
max        169.509933
Name: LifeSquare, dtype: float64

Обработка выбросов в признаке **KitchenSquare**

In [2262]:
train['KitchenSquare'].describe()

count    10000.000000
mean         6.273300
std         28.561113
min          0.000000
25%          1.000000
50%          6.000000
75%          9.000000
max       2014.000000
Name: KitchenSquare, dtype: float64

In [2263]:
KitchenSquare_max_value = np.quantile(train['KitchenSquare'],q=0.999)
KitchenSquare_max_value

72.0010000000002

In [2264]:
KitchenSquare_min_value = np.quantile(train['KitchenSquare'],q=0.01)
KitchenSquare_min_value

0.0

In [2265]:
condition = (train['KitchenSquare'] > KitchenSquare_max_value) | (train['KitchenSquare'] < KitchenSquare_min_value)
train[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1064,14656,62,1.0,47.100719,46.447960,2014.0,4,1.0,2014,0.072158,B,B,2,629,1,1142.904419,0,0,A,108337.484375
1455,12507,54,2.0,79.810532,79.578964,78.0,10,15.0,2014,0.006076,B,B,30,5285,0,645.000000,6,6,B,438708.718750
2916,12390,72,3.0,97.490677,97.490677,96.0,22,25.0,2019,0.210473,B,B,11,2398,2,1994.000000,3,0,B,445074.968750
2969,7441,62,3.0,114.734474,112.589081,112.0,3,3.0,2015,0.072158,B,B,2,629,1,1142.904419,0,0,A,315245.531250
4651,12552,58,3.0,116.405693,113.109650,112.0,3,3.0,2016,0.437885,B,B,23,5735,3,1084.000000,0,5,B,296165.937500
5149,13703,42,1.0,38.071693,19.723547,73.0,9,10.0,2006,0.158249,B,B,21,5731,0,1142.904419,1,0,B,160488.031250
6240,4966,96,3.0,78.445679,47.146755,75.0,9,12.0,1977,0.041116,B,B,53,14892,4,1142.904419,1,4,B,290354.468750
7088,6569,27,1.0,38.220257,18.716856,84.0,4,17.0,2018,0.011654,B,B,4,915,0,1142.904419,0,0,B,99079.960938
8584,14679,81,1.0,32.276665,19.278395,1970.0,6,1.0,1977,0.006076,B,B,30,5285,0,645.000000,6,6,B,105539.554688
9895,2737,27,3.0,123.430069,123.430069,123.0,5,10.0,2015,0.017647,B,B,2,469,0,1142.904419,0,0,B,234194.843750


In [2266]:
train.loc[condition, 'KitchenSquare'] = train['KitchenSquare'].median()
train.KitchenSquare.describe()

count    10000.000000
mean         5.805600
std          4.502124
min          0.000000
25%          1.000000
50%          6.000000
75%          9.000000
max         72.000000
Name: KitchenSquare, dtype: float64

Обработка выбросов в признаке **HouseFloor**

In [2267]:
train.HouseFloor.describe()

count    10000.000000
mean        12.609400
std          6.776192
min          0.000000
25%          9.000000
50%         13.000000
75%         17.000000
max        117.000000
Name: HouseFloor, dtype: float64

In [2268]:
condition = (train.HouseFloor < 1)
train[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
7,11993,74,2.0,80.312927,53.051064,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,1142.904419,0,2,B,221244.15625
23,6641,54,3.0,118.907608,78.544952,0.0,2,0.0,1977,0.006076,B,B,30,5285,0,645.000000,6,6,B,571069.06250
26,4378,27,3.0,106.958870,33.583183,0.0,17,0.0,2018,0.072158,B,B,2,629,1,1142.904419,0,0,A,337299.87500
39,9371,23,2.0,60.503250,39.965691,0.0,16,0.0,1977,0.034656,B,B,0,168,0,1142.904419,0,0,B,229778.06250
44,10521,38,3.0,104.211395,104.211395,0.0,20,0.0,2017,0.060753,B,B,15,2787,2,520.000000,0,7,B,435462.06250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,12302,30,1.0,46.072914,46.072914,0.0,4,0.0,1977,0.000078,B,B,22,6398,141,1046.000000,3,23,B,161403.09375
9769,9384,59,3.0,98.607330,65.135513,0.0,14,0.0,1977,0.019509,B,B,37,7687,11,176.000000,5,5,B,492700.25000
9878,11441,62,2.0,71.985336,71.985336,0.0,5,0.0,1977,0.072158,B,B,2,629,1,1142.904419,0,0,A,158841.62500
9908,3171,62,1.0,35.873962,23.696707,0.0,25,0.0,1977,0.072158,B,B,2,629,1,1142.904419,0,0,A,134379.12500


In [2269]:
train.loc[condition, 'HouseFloor'] = train['Floor']

In [2270]:
condition = (train.HouseFloor < train.Floor)
train[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
14,10953,27,1.0,53.769096,52.408028,1.0,5,4.0,1977,0.072158,B,B,2,629,1,1142.904419,0,0,A,140647.562500
16,2119,27,2.0,49.360649,31.993963,5.0,6,5.0,1983,0.051815,B,B,5,1227,0,1142.904419,0,0,B,117000.382812
21,11935,27,2.0,64.711838,42.745693,1.0,15,1.0,1977,0.211401,B,B,9,1892,0,1142.904419,0,1,B,127200.023438
35,6486,200,3.0,85.280388,58.447968,9.0,6,5.0,1960,0.000000,B,B,33,7425,1,1142.904419,2,5,B,402871.906250
51,10103,94,1.0,35.280895,23.354176,6.0,11,9.0,1971,0.282798,B,B,33,8667,2,1142.904419,0,6,B,148862.203125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9984,2237,52,3.0,64.074974,47.633427,6.0,11,9.0,1983,0.371149,B,B,34,7065,1,750.000000,2,5,B,226434.875000
9985,16159,27,1.0,38.968338,25.740709,1.0,10,1.0,1977,0.211401,B,B,9,1892,0,1142.904419,0,1,B,95529.570312
9989,4145,96,3.0,90.055229,47.860229,11.0,26,24.0,2005,0.041116,B,B,53,14892,4,1142.904419,1,4,B,441108.906250
9991,11553,3,3.0,101.622795,51.628780,12.0,18,17.0,2000,0.265089,B,B,37,5288,0,1937.000000,3,2,B,212883.500000


In [2271]:
train.loc[condition, 'HouseFloor'], train.loc[condition, 'Floor'] = train['Floor'], train['HouseFloor']

In [2272]:
train.HouseYear.describe()

count    1.000000e+04
mean     3.990166e+03
std      2.005003e+05
min      1.910000e+03
25%      1.974000e+03
50%      1.977000e+03
75%      2.001000e+03
max      2.005201e+07
Name: HouseYear, dtype: float64

In [2273]:
condition = (train.HouseYear > 2021)
train[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1497,10814,109,1.0,37.265072,20.239714,9.0,9.0,12.0,20052011,0.136330,B,B,30,6141,10,262.0,3,6,B,254084.531250
4189,11607,147,2.0,44.791836,28.360394,5.0,4.0,9.0,4968,0.319809,B,B,25,4756,16,2857.0,5,8,B,243028.609375


In [2274]:
X_test.loc[condition, 'HouseYear'] = X_test['HouseYear'].median()

In [2275]:
train.HouseYear.describe()

count    1.000000e+04
mean     3.990166e+03
std      2.005003e+05
min      1.910000e+03
25%      1.974000e+03
50%      1.977000e+03
75%      2.001000e+03
max      2.005201e+07
Name: HouseYear, dtype: float64

**Отбор и построение новых признаков**

Количественные переменные.

In [2276]:
#Доля жилой площади LifeSquare в общей площади Square
train['LifeSquare_share'] = train['LifeSquare'] / train['Square'] 

In [2277]:
train.LifeSquare_share.describe()

count    10000.000000
mean         0.648597
std          0.159819
min          0.013893
25%          0.574907
50%          0.652082
75%          0.667856
max          1.000000
Name: LifeSquare_share, dtype: float64

Категориальные переменные

In [2278]:
train.replace({'Ecology_2': {'A':0, 'B':1}, 'Ecology_3': {'A':0, 'B':1}, 'Shops_2': {'A':0, 'B':1}}, inplace = True)

In [2279]:
X = train.drop('Price', axis = 1)
target = train['Price']
y = pd.DataFrame(target, columns=["Price"])

test

Обработка выбросов в признаке **Rooms**

In [2280]:
X_test['Rooms'].describe()

count    5000.000000
mean        1.910000
std         0.838587
min         0.000000
25%         1.000000
50%         2.000000
75%         2.000000
max        17.000000
Name: Rooms, dtype: float64

In [2281]:
Rooms_max_value = np.quantile(X_test['Rooms'],q=0.999)
Rooms_max_value

5.0

In [2282]:
Rooms_min_value = np.quantile(X_test['Rooms'],q=0.001)
Rooms_min_value

1.0

In [2283]:
condition = (X_test['Rooms'] > Rooms_max_value) | (X_test['Rooms'] < Rooms_min_value)
X_test[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
2071,10793,23,6.0,110.750229,70.941124,0.0,2,2.0,2015.0,0.014073,B,B,2,475,0,1146.657227,0,0,B
2406,3343,58,0.0,116.824203,113.692421,0.0,3,3.0,1977.0,0.437885,B,B,23,5735,3,1084.000000,0,5,B
2524,10729,27,0.0,76.345154,42.820797,12.0,14,0.0,1977.0,0.017647,B,B,2,469,0,1146.657227,0,0,B
3217,4058,27,6.0,223.453690,104.113556,16.0,2,2.0,2017.0,0.041116,B,B,53,14892,4,1146.657227,1,4,B
3398,1435,111,17.0,52.866108,32.528343,8.0,15,17.0,1987.0,0.093443,B,B,23,4635,5,3300.000000,2,4,B


In [2284]:
X_test.loc[condition, 'Rooms'] = X_test['Rooms'].median()
X_test.Rooms.describe()

count    5000.000000
mean        1.906200
std         0.805935
min         1.000000
25%         1.000000
50%         2.000000
75%         2.000000
max         5.000000
Name: Rooms, dtype: float64

Обработка выбросов в признаке **Square**

In [2285]:
X_test['Square'].describe()

count    5000.000000
mean       56.449501
std        19.092793
min         1.378543
25%        41.906230
50%        52.921339
75%        66.285130
max       223.453690
Name: Square, dtype: float64

In [2286]:
Square_max_value = np.quantile(X_test['Square'],q=0.999)
Square_max_value

168.730781707764

In [2287]:
Square_min_value = np.quantile(X_test['Square'],q=0.001)
Square_min_value

4.9650765013694755

In [2288]:
condition = (X_test['Square'] > Square_max_value) | (X_test['Square'] < Square_min_value)
X_test[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
66,9011,53,1.0,1.378543,1.353573,1.0,1,1.0,1977.0,0.049637,B,B,34,7759,0,229.000000,1,3,B
608,16401,30,1.0,2.645046,4.338756,1.0,2,1.0,1977.0,0.000078,B,B,22,6398,141,1046.000000,3,23,B
1264,13589,27,3.0,173.974472,76.289474,9.0,3,3.0,2017.0,0.041116,B,B,53,14892,4,1146.657227,1,4,B
1499,7769,124,4.0,189.679581,114.708672,4.0,19,40.0,2013.0,0.174739,B,B,17,3379,9,100.000000,0,3,B
2039,3357,27,3.0,170.476318,75.973610,8.0,2,2.0,2017.0,0.041116,B,B,53,14892,4,1146.657227,1,4,B
3217,4058,27,2.0,223.453690,104.113556,16.0,2,2.0,2017.0,0.041116,B,B,53,14892,4,1146.657227,1,4,B
3909,1397,76,3.0,221.138763,118.055344,4.0,32,48.0,2011.0,0.000000,B,B,7,1660,39,1786.000000,1,5,B
4490,1165,27,1.0,2.372101,1.899119,1.0,2,17.0,1977.0,0.011654,B,B,4,915,0,1146.657227,0,0,B
4713,170,6,2.0,2.900586,61.468563,1.0,18,17.0,2014.0,0.243205,B,B,5,1564,0,540.000000,0,0,B
4902,1420,45,1.0,1.975769,2.900371,1.0,1,1.0,1977.0,0.195781,B,B,23,5212,6,1146.657227,3,2,B


In [2289]:
X_test.loc[condition, 'Square'] = X_test['Square'].median()
X_test.Square.describe()

count    5000.000000
mean       56.357346
std        18.484938
min         4.967143
25%        41.926032
50%        52.921341
75%        66.261539
max       168.729034
Name: Square, dtype: float64

Обработка выбросов в признаке **LifeSquare**

In [2290]:
X_test['LifeSquare'].describe()

count    5000.000000
mean       36.692112
std        16.837259
min         0.333490
25%        24.677814
50%        33.707731
75%        45.110224
max       303.071106
Name: LifeSquare, dtype: float64

In [2291]:
LifeSquare_max_value = np.quantile(X_test['LifeSquare'],q=0.999)
LifeSquare_max_value

128.23075541687024

In [2292]:
LifeSquare_min_value = np.quantile(X_test['LifeSquare'],q=0.001)
LifeSquare_min_value

1.2982615854740143

In [2293]:
condition = (X_test['LifeSquare'] > LifeSquare_max_value) | (X_test['LifeSquare'] < LifeSquare_min_value)
X_test[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
78,6553,34,5.0,122.375275,0.333490,0.0,9,0.0,1977.0,0.069753,B,B,53,13670,4,1146.657227,1,11,B
326,14959,1,2.0,64.112442,1.181139,1.0,13,17.0,1977.0,0.007122,B,B,1,264,0,1146.657227,0,1,B
410,11533,94,2.0,48.713444,303.071106,6.0,5,12.0,1974.0,0.521867,B,B,25,6149,0,1146.657227,0,0,B
2081,1686,62,3.0,137.653214,136.921097,0.0,4,3.0,1977.0,0.072158,B,B,2,629,1,1146.657227,0,0,A
2115,2198,54,3.0,125.401268,128.913467,1.0,7,1.0,2016.0,0.006076,B,B,30,5285,0,645.000000,6,6,B
2126,16531,93,3.0,136.389389,136.689194,0.0,1,1.0,1977.0,0.004949,B,B,0,186,0,1146.657227,0,0,B
2341,3028,9,1.0,43.348347,0.809265,0.0,16,17.0,2015.0,0.161532,B,B,25,5648,1,30.000000,2,4,B
2398,13014,53,3.0,81.141640,0.863989,0.0,10,1.0,2017.0,0.049637,B,B,34,7759,0,229.000000,1,3,B
2557,16053,17,4.0,168.729034,169.901703,0.0,4,0.0,2013.0,0.093443,B,B,23,4635,5,3300.000000,2,4,B
2992,6406,27,2.0,66.469513,0.567267,0.0,4,0.0,2018.0,0.017647,B,B,2,469,0,1146.657227,0,0,B


In [2294]:
X_test.loc[condition, 'LifeSquare'] = X_test['LifeSquare'].median()
X_test.LifeSquare.describe()

count    5000.000000
mean       36.583679
std        16.085766
min         1.298379
25%        24.710508
50%        33.707733
75%        45.056705
max       128.230072
Name: LifeSquare, dtype: float64

In [2295]:
condition = (X_test.LifeSquare > X_test.Square)
X_test.loc[condition, 'LifeSquare'], X_test.loc[condition, 'Square'] = X_test['Square'], X_test['LifeSquare']

In [2296]:
X_test.LifeSquare.describe()

count    5000.000000
mean       36.459442
std        15.853214
min         1.298379
25%        24.710508
50%        33.707733
75%        45.022210
max       126.525032
Name: LifeSquare, dtype: float64

Обработка выбросов в признаке **KitchenSquare**

In [2297]:
X_test['KitchenSquare'].describe()

count    5000.000000
mean        5.976800
std         9.950147
min         0.000000
25%         1.000000
50%         6.000000
75%         9.000000
max       620.000000
Name: KitchenSquare, dtype: float64

In [2298]:
KitchenSquare_max_value = np.quantile(X_test['KitchenSquare'],q=0.999)
KitchenSquare_max_value

61.001000000000204

In [2299]:
KitchenSquare_min_value = np.quantile(X_test['KitchenSquare'],q=0.01)
KitchenSquare_min_value

0.0

In [2300]:
condition = (X_test['KitchenSquare'] > KitchenSquare_max_value) | (X_test['KitchenSquare'] < KitchenSquare_min_value)
X_test[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
1456,5260,73,3.0,69.358246,51.247581,65.0,6,6.0,1931.0,0.042032,B,B,37,6856,84,1940.000000,2,5,B
1777,3341,62,3.0,112.114021,112.114021,112.0,3,3.0,2017.0,0.072158,B,B,2,629,1,1146.657227,0,0,A
1964,14594,11,2.0,42.795303,24.223770,620.0,11,14.0,1972.0,0.038693,B,B,28,6533,1,1015.000000,2,5,B
4281,8015,27,1.0,66.099098,33.639610,62.0,3,7.0,2016.0,0.014058,B,B,1,290,0,1146.657227,0,0,B
4555,12640,6,2.0,54.629143,31.486307,97.0,4,17.0,2015.0,0.243205,B,B,5,1564,0,540.000000,0,0,B


In [2301]:
X_test.loc[condition, 'KitchenSquare'] = X_test['KitchenSquare'].median()
X_test.KitchenSquare.describe()

count    5000.000000
mean        5.791600
std         4.279959
min         0.000000
25%         1.000000
50%         6.000000
75%         9.000000
max        61.000000
Name: KitchenSquare, dtype: float64

Обработка выбросов в признаке **HouseFloor**

In [2302]:
train.HouseFloor.describe()

count    10000.000000
mean        13.435700
std          6.094035
min          1.000000
25%          9.000000
50%         14.000000
75%         17.000000
max        117.000000
Name: HouseFloor, dtype: float64

In [2303]:
condition = (X_test.HouseFloor < 1)
X_test[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
22,12694,27,3.0,83.670029,46.358356,0.0,8,0.0,2015.0,0.072158,B,B,2,629,1,1146.657227,0,0,A
24,8968,27,2.0,69.849243,44.741970,0.0,3,0.0,1977.0,0.011654,B,B,4,915,0,1146.657227,0,0,B
30,2982,6,2.0,63.460686,40.649776,0.0,13,0.0,1977.0,0.243205,B,B,5,1564,0,540.000000,0,0,B
32,2449,1,1.0,66.426582,42.549583,0.0,14,0.0,1977.0,0.007122,B,B,1,264,0,1146.657227,0,1,B
46,12309,45,1.0,42.899570,27.479343,0.0,25,0.0,1977.0,0.195781,B,B,23,5212,6,1146.657227,3,2,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4856,10262,27,2.0,69.196442,44.323818,0.0,10,0.0,1977.0,0.017647,B,B,2,469,0,1146.657227,0,0,B
4887,12116,88,3.0,89.549561,57.361019,0.0,20,0.0,1977.0,0.127376,B,B,43,8429,3,1146.657227,3,9,B
4892,16357,58,1.0,49.205978,31.518917,0.0,4,0.0,1977.0,0.437885,B,B,23,5735,3,1084.000000,0,5,B
4976,4723,95,1.0,45.542938,29.172554,0.0,6,0.0,1977.0,0.000699,B,B,14,3369,24,4129.000000,0,3,B


In [2304]:
X_test.loc[condition, 'HouseFloor'] = X_test['Floor']

In [2305]:
condition = (X_test.HouseFloor < X_test.Floor)
X_test[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
1,15856,74,2.0,69.263184,44.366570,1.0,6,1.0,1977.0,0.075779,B,B,6,1437,3,1146.657227,0,2,B
10,15975,61,3.0,64.543655,42.651543,7.0,10,9.0,1981.0,0.300323,B,B,52,10311,6,1146.657227,1,9,B
14,5214,27,1.0,37.555195,24.056002,1.0,5,1.0,1977.0,0.211401,B,B,9,1892,0,1146.657227,0,1,B
27,11160,58,1.0,48.610661,48.610661,1.0,4,3.0,1977.0,0.437885,B,B,23,5735,3,1084.000000,0,5,B
36,9348,205,1.0,37.480812,16.851795,8.0,6,5.0,1933.0,0.169091,B,B,19,3856,10,1146.657227,2,5,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4968,15181,27,4.0,104.664207,66.903564,1.0,4,1.0,2016.0,0.017647,B,B,2,469,0,1146.657227,0,0,B
4972,16064,30,1.0,38.376236,37.003311,1.0,2,1.0,1977.0,0.000078,B,B,22,6398,141,1046.000000,3,23,B
4989,2591,39,2.0,44.203590,30.191690,6.0,10,9.0,1972.0,0.428826,B,B,25,5011,8,730.000000,0,2,B
4993,14562,113,2.0,46.627884,27.058739,5.0,6,5.0,1969.0,0.307467,B,A,30,5048,9,325.000000,2,5,B


In [2306]:
X_test.loc[condition, 'HouseFloor'], X_test.loc[condition, 'Floor'] = X_test['Floor'], X_test['HouseFloor']

In [2307]:
train.HouseYear.describe()

count    1.000000e+04
mean     3.990166e+03
std      2.005003e+05
min      1.910000e+03
25%      1.974000e+03
50%      1.977000e+03
75%      2.001000e+03
max      2.005201e+07
Name: HouseYear, dtype: float64

In [2308]:
condition = (train.HouseYear > 2021)
train[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_share
1497,10814,109,1.0,37.265072,20.239714,9.0,9.0,12.0,20052011,0.136330,...,1,30,6141,10,262.0,3,6,1,254084.531250,0.543128
4189,11607,147,2.0,44.791836,28.360394,5.0,4.0,9.0,4968,0.319809,...,1,25,4756,16,2857.0,5,8,1,243028.609375,0.633160


In [2309]:
X_test.loc[condition, 'HouseYear'] = X_test['HouseYear'].median()

In [2310]:
train.HouseYear.describe()

count    1.000000e+04
mean     3.990166e+03
std      2.005003e+05
min      1.910000e+03
25%      1.974000e+03
50%      1.977000e+03
75%      2.001000e+03
max      2.005201e+07
Name: HouseYear, dtype: float64

**Отбор и построение новых признаков**

Количественные переменные.

In [2311]:
#Доля жилой площади LifeSquare в общей площади Square
X_test['LifeSquare_share'] = X_test['LifeSquare'] / X_test['Square'] 

In [2312]:
X_test.LifeSquare_share.describe()

count    5000.000000
mean        0.644575
std         0.159857
min         0.015141
25%         0.577323
50%         0.640551
75%         0.669450
max         1.000000
Name: LifeSquare_share, dtype: float64

Категориальные переменные

In [2313]:
X_test.replace({'Ecology_2': {'A':0, 'B':1}, 'Ecology_3': {'A':0, 'B':1}, 'Shops_2': {'A':0, 'B':1}}, inplace = True)

# Базовая модель

In [2314]:
#разбиваем на тренировочную и валидационную
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X.set_index('Id'), y, test_size=0.2, random_state=42)

In [2315]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 200, max_depth = 10, random_state = 42)


In [2316]:
model.fit(X_train, y_train.values[:,0])

RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42)

In [2317]:
predict_train = model.predict(X_train)
print(r2_score(y_train, predict_train))
predict_valid = model.predict(X_valid)
print(r2_score(y_valid, predict_valid))

0.8502895361970093
0.699778732413322


In [2318]:
predict_test = model.predict(X_test.set_index('Id'))

In [2319]:
X_test['Price'] = predict_test
X_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,LifeSquare_share,Price
0,725,58,2.0,49.882645,33.432781,6.0,6.0,14.0,1972.0,0.310199,...,1,11,2748,1,1146.657227,0,0,1,0.670229,160029.467874
1,15856,74,2.0,69.263184,44.366570,1.0,6.0,6.0,1977.0,0.075779,...,1,6,1437,3,1146.657227,0,2,1,0.640551,208179.949533
2,5480,190,1.0,13.597818,13.597818,12.0,2.0,5.0,1909.0,0.000000,...,1,30,7538,87,4702.000000,5,5,1,1.000000,193121.228573
3,15664,47,2.0,73.046608,51.940842,9.0,22.0,22.0,2007.0,0.101872,...,1,23,4583,3,1146.657227,3,3,1,0.711064,341411.000104
4,14275,27,1.0,47.527111,43.387569,1.0,17.0,17.0,2017.0,0.072158,...,1,2,629,1,1146.657227,0,0,0,0.912901,138173.733468


In [2320]:
X_test[['Id', 'Price']].to_csv('OLevshina_predictions.csv', index=False)